# HALO Synthetic Data Generation for MIMIC-III

This notebook trains the HALO (Hierarchical Autoregressive Language mOdel) on your MIMIC-III data and generates synthetic patients.

## What You'll Need

1. **MIMIC-III Access**: Download these files from PhysioNet:
   - `ADMISSIONS.csv`
   - `DIAGNOSES_ICD.csv`

2. **Google Colab**: Free tier works, but GPU recommended (Runtime → Change runtime type → GPU)

3. **Time**:
   - Demo (5 epochs, 1K samples): ~20-30 min on GPU
   - Production (80 epochs, 10K samples): ~6-10 hours on GPU

## How It Works

1. **Setup**: Install PyHealth and mount Google Drive
2. **Upload Data**: Upload your MIMIC-III CSV files
3. **Configure**: Set hyperparameters (epochs, batch size, etc.)
4. **Train**: Train HALO model (checkpoints saved to Drive)
5. **Generate**: Create synthetic patients using trained model
6. **Download**: Get CSV file with synthetic data

## Important Notes

⚠️ **Colab Timeout**: Free Colab sessions timeout after 12 hours. For production training (80 epochs), consider:
- Colab Pro for longer sessions
- Running on your own GPU cluster using `examples/slurm/train_halo_mimic3.slurm`

📊 **Demo vs Production**:
- Demo defaults (5 epochs, 1K samples) let you test the pipeline quickly
- Production settings (80 epochs, 10K samples) match the published HALO results

## References

- [HALO Paper](https://arxiv.org/abs/2406.16061)
- [PyHealth Documentation](https://pyhealth.readthedocs.io/)
- [MIMIC-III Access](https://physionet.org/content/mimiciii/)

---
# 1. Setup & Installation

In [ ]:
import subprocess
import sys
from datetime import datetime

# Install PyHealth from GitHub (gets latest HALO implementation)
# For development/CI: set BRANCH to specific branch name (e.g., 'halo-pr-528')
# For production: leave BRANCH as None to use main branch
FORK = 'jalengg'
BRANCH = 'halo-pr-528'  # Change to 'halo-pr-528' or other branch name for development

if BRANCH:
    install_url = f"git+https://github.com/{FORK}/PyHealth.git@{BRANCH}"
    print(f"Installing PyHealth from branch '{BRANCH}'...")
else:
    install_url = f"git+https://github.com/{FORK}/PyHealth.git"
    print("Installing PyHealth from main branch...")

# Install with error checking (--no-cache-dir ensures latest version)
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "--no-cache-dir", install_url],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✓ PyHealth installed successfully! [{datetime.now().strftime('%Y%m%d%H%M%S')}]")
else:
    print("❌ PyHealth installation failed!")
    print(f"
Error output:")
    print(result.stderr)
    raise RuntimeError("PyHealth installation failed. Please check the error above.")


In [ ]:
# Import required libraries
import os
import sys
import torch
import pickle
import pandas as pd
import shutil
from google.colab import drive, files
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

print("✓ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Mount Google Drive for persistent storage
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✓ Google Drive mounted at /content/drive")

# Create directory structure in Drive
base_dir = '/content/drive/MyDrive/HALO_Training'
data_dir = f'{base_dir}/data'
checkpoint_dir = f'{base_dir}/checkpoints'
pkl_data_dir = f'{base_dir}/pkl_data'
output_dir = f'{base_dir}/output'

for dir_path in [base_dir, data_dir, checkpoint_dir, pkl_data_dir, output_dir]:
    os.makedirs(dir_path, exist_ok=True)

print(f"\n✓ Directory structure created:")
print(f"  Base: {base_dir}")
print(f"  Data: {data_dir}")
print(f"  Checkpoints: {checkpoint_dir}")
print(f"  Vocabulary: {pkl_data_dir}")
print(f"  Output: {output_dir}")

---
# 2. Configuration

Configure your training and generation parameters below.

**For Quick Demo (recommended first time):**
- Leave defaults (5 epochs, 1K samples)
- Runs in ~20-30 minutes on GPU

**For Production Quality:**
- Set `EPOCHS = 80`
- Set `N_SYNTHETIC_SAMPLES = 10000`
- Expect ~6-10 hours on GPU

In [ ]:
# ============================================
# CONFIGURATION - Modify these parameters
# ============================================

# Training configuration
EPOCHS = 5                  # Demo: 5, Production: 80
BATCH_SIZE = 32            # Demo: 32, Production: 48
LEARNING_RATE = 0.0001     # Standard HALO learning rate

# Generation configuration
N_SYNTHETIC_SAMPLES = 1000  # Demo: 1000, Production: 10000

# Advanced HALO model configuration (usually don't need to change)
N_POSITIONS = 56           # Maximum sequence length
N_CTX = 48                 # Context window
N_EMBD = 768              # Embedding dimension
N_LAYER = 12              # Number of transformer layers
N_HEAD = 12               # Number of attention heads

# Display configuration
print("=" * 60)
print("HALO CONFIGURATION")
print("=" * 60)
print(f"Training:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"\nGeneration:")
print(f"  Synthetic samples: {N_SYNTHETIC_SAMPLES}")
print(f"\nModel architecture:")
print(f"  Embedding dim: {N_EMBD}")
print(f"  Layers: {N_LAYER}")
print(f"  Attention heads: {N_HEAD}")
print("=" * 60)

# Estimate runtime
if torch.cuda.is_available():
    runtime_min = EPOCHS * 4  # Rough estimate: 4 min per epoch on GPU
    print(f"\nEstimated training time: ~{runtime_min}-{runtime_min*2} minutes on GPU")
else:
    print(f"\n⚠️  WARNING: No GPU detected! Training will be very slow.")
    print(f"   Go to Runtime → Change runtime type → Select GPU")

---
# 3. Data Upload

Upload your MIMIC-III CSV files. You need these 2 files:

1. `ADMISSIONS.csv` - Patient admission records
2. `DIAGNOSES_ICD.csv` - Diagnosis codes (ICD-9)

**Note**: Files will be saved to Google Drive and persist across Colab sessions. If files already exist in Drive, you can skip uploading and reuse them.

In [ ]:
import os

# Required MIMIC-III files (only 2 needed for HALO)
required_files = {
    'ADMISSIONS.csv': 'Patient admission records',
    'DIAGNOSES_ICD.csv': 'Diagnosis codes (ICD-9)'
}

# Check if files already exist in Google Drive
existing_files = {f: os.path.exists(f'{data_dir}/{f}') for f in required_files}
missing_files = [f for f, exists in existing_files.items() if not exists]

if not missing_files:
    print("✓ All required files already exist in Google Drive!")
    for filename in required_files:
        file_path = f'{data_dir}/{filename}'
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  ✓ {filename} ({file_size_mb:.1f} MB)")
    print(f"\nSkipping upload. To re-upload, delete files from: {data_dir}")
else:
    print("Upload the following MIMIC-III files:")
    for i, filename in enumerate(missing_files, 1):
        print(f"  {i}. {filename} - {required_files[filename]}")
    print("\nYou can upload files one at a time or all together.\n")
    
    uploaded = files.upload()
    
    # Normalize filenames - handle Colab's automatic renaming (e.g., "ADMISSIONS (1).csv")
    print("\nProcessing uploaded files...")
    for uploaded_name in uploaded.keys():
        # Find which required file this matches
        matched_file = None
        for req_file in required_files:
            # Extract base name without extension
            req_base = req_file.replace('.csv', '')
            # Check if uploaded name contains the required base name
            if req_base in uploaded_name and uploaded_name.endswith('.csv'):
                matched_file = req_file
                break
        
        if matched_file:
            src = f'/content/{uploaded_name}'
            dst = f'{data_dir}/{matched_file}'
            shutil.copy(src, dst)
            file_size_mb = os.path.getsize(dst) / (1024 * 1024)
            print(f"  ✓ {matched_file} ({file_size_mb:.1f} MB)")
            if matched_file in missing_files:
                missing_files.remove(matched_file)
    
    # Check if we still have missing files
    if missing_files:
        print(f"\n⚠️  Still missing: {missing_files}")
        print("Please run this cell again to upload the remaining files.")
    else:
        print("\n✓ All files uploaded successfully!")
        print(f"   Location: {data_dir}")

In [ ]:
# Validate uploaded files
print("Validating data files...")

# Check ADMISSIONS.csv
admissions = pd.read_csv(f'{data_dir}/ADMISSIONS.csv', nrows=5)
print(f"\n✓ ADMISSIONS.csv: {len(admissions.columns)} columns")
print(f"   Sample columns: {', '.join(admissions.columns[:3])}")

# Check DIAGNOSES_ICD.csv
diagnoses = pd.read_csv(f'{data_dir}/DIAGNOSES_ICD.csv', nrows=5)
print(f"\n✓ DIAGNOSES_ICD.csv: {len(diagnoses.columns)} columns")
print(f"   Sample columns: {', '.join(diagnoses.columns[:3])}")

print("\n" + "=" * 60)
print("✓ DATA VALIDATION COMPLETE")
print("=" * 60)

---
# 4. Training

Train the HALO model on your MIMIC-III data.

**What happens during training:**
- Dataset is preprocessed and vocabularies are created
- HALO model trains for the specified number of epochs
- Best checkpoint is automatically saved when validation loss improves
- Progress updates print every 1,000 iterations

**Checkpoints saved to Drive:**
- `halo_model` - Best model (lowest validation loss)
- `halo_model_final` - Final model after all epochs
- Vocabulary files in `pkl_data/`

**Resume capability**: If training is interrupted, checkpoints in Drive allow resuming.

In [ ]:
from pyhealth.datasets.halo_mimic3 import HALO_MIMIC3Dataset
from pyhealth.models.generators.halo import HALO
from pyhealth.models.generators.halo_resources.halo_config import HALOConfig

# Check for existing checkpoint
existing_checkpoint = os.path.exists(f'{checkpoint_dir}/halo_model')
if existing_checkpoint:
    print("=" * 60)
    print("⚠️  EXISTING CHECKPOINT FOUND")
    print("=" * 60)
    print(f"Found checkpoint at: {checkpoint_dir}/halo_model")
    print("\nOptions:")
    print("  1. Skip training and go to generation (recommended if training completed)")
    print("  2. Continue running this cell to retrain (will overwrite checkpoint)")
    print("\nTo skip training, jump to the 'Generation' section below.")
    print("=" * 60)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}", flush=True)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}", flush=True)
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB", flush=True)
else:
    print("⚠️  WARNING: Training on CPU will be very slow!", flush=True)

# Load and preprocess dataset
print(f"\n{'='*60}", flush=True)
print("Loading and preprocessing MIMIC-III dataset...", flush=True)
print(f"{'='*60}", flush=True)
print(f"Data directory: {data_dir}", flush=True)
print(f"Output directory: {checkpoint_dir}", flush=True)

dataset = HALO_MIMIC3Dataset(
    mimic3_dir=data_dir,
    pkl_data_dir=pkl_data_dir,
    gzip=False
)

print(f"\n{'='*60}", flush=True)
print("Dataset preprocessing complete!", flush=True)
print(f"{'='*60}", flush=True)

# Load vocabulary sizes
code_to_index = pickle.load(open(f"{pkl_data_dir}/codeToIndex.pkl", "rb"))
id_to_label = pickle.load(open(f"{pkl_data_dir}/idToLabel.pkl", "rb"))

code_vocab_size = len(code_to_index)
label_vocab_size = len(id_to_label)
special_vocab_size = 3
total_vocab_size = code_vocab_size + label_vocab_size + special_vocab_size

print(f"Vocabulary sizes:", flush=True)
print(f"  Code vocabulary: {code_vocab_size}", flush=True)
print(f"  Label vocabulary: {label_vocab_size}", flush=True)
print(f"  Special tokens: {special_vocab_size}", flush=True)
print(f"  Total vocabulary: {total_vocab_size}", flush=True)

# HALO configuration
print(f"\n{'='*60}", flush=True)
print("Initializing HALO configuration", flush=True)
print(f"{'='*60}", flush=True)

config = HALOConfig(
    total_vocab_size=total_vocab_size,
    code_vocab_size=code_vocab_size,
    label_vocab_size=label_vocab_size,
    special_vocab_size=special_vocab_size,
    n_positions=N_POSITIONS,
    n_ctx=N_CTX,
    n_embd=N_EMBD,
    n_layer=N_LAYER,
    n_head=N_HEAD,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    batch_size=BATCH_SIZE,
    sample_batch_size=256,
    epoch=EPOCHS,
    pos_loss_weight=None,
    lr=LEARNING_RATE
)

print("Configuration:", flush=True)
print(f"  Embedding dim: {config.n_embd}", flush=True)
print(f"  Layers: {config.n_layer}", flush=True)
print(f"  Attention heads: {config.n_head}", flush=True)
print(f"  Batch size: {config.batch_size}", flush=True)
print(f"  Epochs: {config.epoch}", flush=True)
print(f"  Learning rate: {config.lr}", flush=True)

# Train HALO model
print(f"\n{'='*60}", flush=True)
print("Training HALO model...", flush=True)
print(f"{'='*60}", flush=True)
print(f"Training for {EPOCHS} epochs", flush=True)
print(f"Progress updates every 1,000 iterations", flush=True)
print(f"Checkpoints saved when validation loss improves", flush=True)
print(f"{'='*60}\n", flush=True)

model = HALO(
    dataset=dataset,
    config=config,
    save_dir=checkpoint_dir,
    train_on_init=True
)

print(f"\n{'='*60}", flush=True)
print("✓ TRAINING COMPLETE!", flush=True)
print(f"{'='*60}", flush=True)

# Save final checkpoint
final_state = {
    'model': model.model.state_dict(),
    'optimizer': model.optimizer.state_dict(),
    'iteration': 'final',
    'epoch': config.epoch
}
torch.save(final_state, os.path.join(checkpoint_dir, 'halo_model_final'))
print(f"✓ Final checkpoint saved to: {checkpoint_dir}/halo_model_final", flush=True)

# Copy best checkpoint
best_path = os.path.join(checkpoint_dir, 'halo_model')
if os.path.exists(best_path):
    shutil.copy(best_path, os.path.join(checkpoint_dir, 'halo_model_best'))
    print(f"✓ Best checkpoint copied to: {checkpoint_dir}/halo_model_best", flush=True)

print(f"\n✓ Training artifacts saved:", flush=True)
print(f"  - Checkpoints: {checkpoint_dir}", flush=True)
print(f"  - Vocabulary: {pkl_data_dir}", flush=True)

---
# 5. Generation

Generate synthetic patients using the trained HALO model.

**What happens during generation:**
- Loads the best checkpoint from training
- Generates the specified number of synthetic patients
- Each patient has multiple visits with ICD-9 diagnosis codes
- Outputs CSV file with columns: SUBJECT_ID, VISIT_NUM, ICD9_CODE

**Generation time**:
- 1,000 patients: ~10-15 minutes
- 10,000 patients: ~1-2 hours

In [ ]:
from pyhealth.models.generators.halo_resources.halo_model import HALOModel

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}", flush=True)

# Load vocabulary and configuration
print(f"\n{'='*60}", flush=True)
print(f"Loading vocabulary from {pkl_data_dir}", flush=True)
print(f"{'='*60}", flush=True)

code_to_index = pickle.load(open(f"{pkl_data_dir}/codeToIndex.pkl", "rb"))
index_to_code = pickle.load(open(f"{pkl_data_dir}/indexToCode.pkl", "rb"))
id_to_label = pickle.load(open(f"{pkl_data_dir}/idToLabel.pkl", "rb"))
train_dataset = pickle.load(open(f"{pkl_data_dir}/trainDataset.pkl", "rb"))

code_vocab_size = len(code_to_index)
label_vocab_size = len(id_to_label)
special_vocab_size = 3
total_vocab_size = code_vocab_size + label_vocab_size + special_vocab_size

print(f"Vocabulary sizes:", flush=True)
print(f"  Code vocabulary: {code_vocab_size}", flush=True)
print(f"  Label vocabulary: {label_vocab_size}", flush=True)
print(f"  Total vocabulary: {total_vocab_size}", flush=True)

# Create config (same as training)
config = HALOConfig(
    total_vocab_size=total_vocab_size,
    code_vocab_size=code_vocab_size,
    label_vocab_size=label_vocab_size,
    special_vocab_size=special_vocab_size,
    n_positions=N_POSITIONS,
    n_ctx=N_CTX,
    n_embd=N_EMBD,
    n_layer=N_LAYER,
    n_head=N_HEAD,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    batch_size=BATCH_SIZE,
    sample_batch_size=256,
    epoch=EPOCHS,
    pos_loss_weight=None,
    lr=LEARNING_RATE
)

# Create minimal dataset object for interface compatibility
class MinimalDataset:
    def __init__(self, pkl_data_dir):
        self.pkl_data_dir = pkl_data_dir

dataset = MinimalDataset(pkl_data_dir)

# Load trained model
checkpoint_path = f'{checkpoint_dir}/halo_model'
if not os.path.exists(checkpoint_path):
    checkpoint_path = f'{checkpoint_dir}/halo_model_best'
if not os.path.exists(checkpoint_path):
    checkpoint_path = f'{checkpoint_dir}/halo_model_final'

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(
        f"No checkpoint found! Expected one of:\n"
        f"  {checkpoint_dir}/halo_model\n"
        f"  {checkpoint_dir}/halo_model_best\n"
        f"  {checkpoint_dir}/halo_model_final\n"
        f"Please run the Training section first."
    )

print(f"\n{'='*60}", flush=True)
print(f"Loading checkpoint from {checkpoint_path}", flush=True)
print(f"{'='*60}", flush=True)

halo_model = HALOModel(config).to(device)
checkpoint = torch.load(checkpoint_path, map_location=device)
halo_model.load_state_dict(checkpoint['model'])
halo_model.eval()

print("✓ Model loaded successfully", flush=True)

# Generate synthetic patients
print(f"\n{'='*60}", flush=True)
print(f"Generating {N_SYNTHETIC_SAMPLES} synthetic patients...", flush=True)
print(f"{'='*60}", flush=True)

# Estimate time
if torch.cuda.is_available():
    est_minutes = max(10, N_SYNTHETIC_SAMPLES // 100)
    print(f"Estimated time: ~{est_minutes} minutes on GPU", flush=True)
print(f"", flush=True)

# Create HALO instance for generation
halo = HALO(
    dataset=dataset,
    config=config,
    save_dir=checkpoint_dir,
    train_on_init=False
)
halo.model = halo_model
halo.train_ehr_dataset = train_dataset[:N_SYNTHETIC_SAMPLES]
halo.index_to_code = index_to_code

# Generate synthetic data (creates pickle file)
halo.synthesize_dataset(pkl_save_dir=output_dir + "/")

# Load generated pickle
generated_pkl = os.path.join(output_dir, "haloDataset.pkl")
synthetic_data = pickle.load(open(generated_pkl, "rb"))

print(f"\n{'='*60}", flush=True)
print("✓ GENERATION COMPLETE!", flush=True)
print(f"{'='*60}", flush=True)

# Print statistics
print(f"\nSynthetic data statistics:", flush=True)
print(f"  Total patients: {len(synthetic_data)}", flush=True)

total_visits = sum(len(p['visits']) for p in synthetic_data)
avg_visits = total_visits / len(synthetic_data)
print(f"  Total visits: {total_visits}", flush=True)
print(f"  Avg visits per patient: {avg_visits:.2f}", flush=True)

total_codes = sum(len(code) for p in synthetic_data for visit in p['visits'] for code in [visit])
avg_codes = total_codes / total_visits if total_visits > 0 else 0
print(f"  Avg codes per visit: {avg_codes:.2f}", flush=True)

In [ ]:
# Convert to CSV format
print(f"\n{'='*60}", flush=True)
print("Converting to CSV format...", flush=True)
print(f"{'='*60}", flush=True)

csv_path = f'{output_dir}/halo_synthetic_data.csv'

records = []
for patient_idx, patient in enumerate(synthetic_data):
    patient_id = f"SYNTHETIC_{patient_idx+1:06d}"
    for visit_num, visit in enumerate(patient['visits'], 1):
        for code_idx in visit:
            icd9_code = index_to_code[code_idx]
            records.append({
                'SUBJECT_ID': patient_id,
                'VISIT_NUM': visit_num,
                'ICD9_CODE': icd9_code
            })

df = pd.DataFrame(records)
df.to_csv(csv_path, index=False)

print(f"✓ CSV saved to: {csv_path}", flush=True)
print(f"  Total records: {len(df):,}", flush=True)
print(f"  File size: {os.path.getsize(csv_path) / (1024*1024):.2f} MB", flush=True)

# Display sample
print(f"\nFirst 10 rows:", flush=True)
display(df.head(10))

---
# 6. Results & Download

View statistics and download your synthetic data.

**CSV Format**:
- `SUBJECT_ID`: Synthetic patient identifier (SYNTHETIC_000001, etc.)
- `VISIT_NUM`: Visit sequence number (1, 2, 3, ...)
- `ICD9_CODE`: ICD-9 diagnosis code

**Data Quality Checks**:
- Patient IDs should be unique and sequential
- Visit numbers should be sequential for each patient
- ICD-9 codes should match MIMIC-III code format

In [ ]:
# Validate generated data
print("=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Check 1: Patient IDs
unique_patients = df['SUBJECT_ID'].nunique()
print(f"\n✓ Unique patients: {unique_patients}")
assert unique_patients == N_SYNTHETIC_SAMPLES, f"Expected {N_SYNTHETIC_SAMPLES} patients"

# Check 2: No empty values
empty_subjects = df['SUBJECT_ID'].isna().sum()
empty_visits = df['VISIT_NUM'].isna().sum()
empty_codes = df['ICD9_CODE'].isna().sum()

print(f"✓ Empty values check:")
print(f"  Subject IDs: {empty_subjects} (should be 0)")
print(f"  Visit numbers: {empty_visits} (should be 0)")
print(f"  ICD9 codes: {empty_codes} (should be 0)")

assert empty_subjects == 0 and empty_visits == 0 and empty_codes == 0, "Found empty values!"

# Check 3: Visit number sequencing (sample first patient)
first_patient = df[df['SUBJECT_ID'] == 'SYNTHETIC_000001']['VISIT_NUM'].tolist()
is_sequential = first_patient == list(range(1, len(first_patient) + 1))
print(f"\n✓ Visit sequencing (first patient): {is_sequential}")

# Check 4: ICD9 code format (should be strings, common patterns)
sample_codes = df['ICD9_CODE'].head(20).tolist()
print(f"\n✓ Sample ICD9 codes: {', '.join(map(str, sample_codes[:10]))}")

# Check 5: Distribution statistics
codes_per_patient = df.groupby('SUBJECT_ID').size()
print(f"\n✓ Codes per patient distribution:")
print(f"  Min: {codes_per_patient.min()}")
print(f"  Max: {codes_per_patient.max()}")
print(f"  Mean: {codes_per_patient.mean():.2f}")
print(f"  Median: {codes_per_patient.median():.2f}")

visits_per_patient = df.groupby('SUBJECT_ID')['VISIT_NUM'].max()
print(f"\n✓ Visits per patient distribution:")
print(f"  Min: {visits_per_patient.min()}")
print(f"  Max: {visits_per_patient.max()}")
print(f"  Mean: {visits_per_patient.mean():.2f}")
print(f"  Median: {visits_per_patient.median():.2f}")

print("\n" + "=" * 60)
print("✓ ALL QUALITY CHECKS PASSED")
print("=" * 60)

In [ ]:
# Download CSV file
print("=" * 60)
print("DOWNLOAD SYNTHETIC DATA")
print("=" * 60)

print(f"\nYour synthetic data is ready:")
print(f"  File: halo_synthetic_data.csv")
print(f"  Patients: {unique_patients:,}")
print(f"  Total records: {len(df):,}")
print(f"  Size: {os.path.getsize(csv_path) / (1024*1024):.2f} MB")

print(f"\nDownloading file to your computer...")
files.download(csv_path)

print(f"\n✓ Download started!")
print(f"\nFile also saved in Google Drive:")
print(f"  {csv_path}")

---
## 🎉 Congratulations!

You've successfully:
1. ✅ Trained a HALO model on your MIMIC-III data
2. ✅ Generated synthetic patients
3. ✅ Validated the synthetic data quality
4. ✅ Downloaded the CSV file

## Next Steps

**Use your synthetic data:**
- Train predictive models (readmission, mortality, etc.)
- Develop clinical decision support tools
- Share data without privacy concerns

**Generate more samples:**
- Change `N_SYNTHETIC_SAMPLES` and re-run generation section
- No need to retrain - the checkpoint is saved!

**Production training:**
- For publication-quality results, train with `EPOCHS = 80`
- Consider using Colab Pro or a dedicated GPU cluster
- See `examples/slurm/train_halo_mimic3.slurm` for cluster usage

## Troubleshooting

**Out of memory errors:**
- Reduce `BATCH_SIZE` (try 24 or 16)
- Close other browser tabs
- Restart runtime and try again

**Training too slow:**
- Verify GPU is enabled (Runtime → Change runtime type)
- Reduce dataset size for testing
- Consider using Colab Pro for better GPUs

**Questions or issues:**
- PyHealth docs: https://pyhealth.readthedocs.io/
- HALO paper: https://arxiv.org/abs/2406.16061
- GitHub issues: https://github.com/sunlabuiuc/PyHealth/issues